# Importação das libs

In [1]:
#!pip3 install investpy
#https://br.advfn.com/indice/ibovespa

In [2]:
# Libs necessarias


# Libs para análise de dados
import pandas as pd
import pandas_datareader.data as web
import numpy as np

# Libs para visualização de dados
import matplotlib.pyplot as plt
import plotly.express as px

# Lib para manipulação de dados
from datetime import datetime

# APIs
import yfinance as yf
yf.pdr_override()
import investpy as inv

# Lib para avisos
import warnings
warnings.filterwarnings('ignore')

# 1. Capturando os tickers das ações do Brasil

In [3]:
df_br = inv.stocks.get_stocks(country = 'brazil')

In [4]:
df_br.head()

,country,name,full_name,isin,currency,symbol
0,brazil,ABC BRASIL PN,Banco ABC Brasil SA,BRABCBACNPR4,BRL,ABCB4
1,brazil,BRASILAGRO ON,BrasilAgro - Co ON NM,BRAGROACNOR7,BRL,AGRO3
2,brazil,RUMO ON NM,RUMO Logistica Operadora Multimodal SA,BRRAILACNOR9,BRL,RAIL3
3,brazil,ALPARGATAS ON,Alpargatas SA,BRALPAACNOR0,BRL,ALPA3
4,brazil,ALPARGATAS PN,Alpargatas SA,BRALPAACNPR7,BRL,ALPA4


**Criando uma coluna de tickers no padrão do yahoo finance**

In [5]:
tickers = df_br['symbol'] + '.SA'

### Selecionando 20 tickers

In [6]:
tickers20 = tickers[:20]
len(tickers20)

20

# 2. Capturando os dados das ações

**Escolhendo o período**

In [7]:
period = '2y'

**Definindo a função de captura das cotações**

In [8]:
def get_stock(name, interval):
    """
    Essa função recebe o código da ação e o intervalo de tempo de interesse para retornar
    um data frame de séries temporais com as cotações.
    """
    stock = web.get_data_yahoo( name, period = interval )
    stock['Return'] = stock['Adj Close'].pct_change()
    stock = stock.dropna()
    return stock

**Criando um dicionários com as cotações de todos os ativos**

In [9]:
stockDict = {}

for ticker in tickers20:
    try:
        stockDict[ticker] = get_stock(ticker, period)
    except:
        print('Não foi possível')


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

1 Failed download:
- BBRK3.SA: No

**Visualizando o sumário dos ativos**

In [10]:
summary = dict(enumerate(stockDict.keys()))
summary

{0: 'ABCB4.SA',
 1: 'AGRO3.SA',
 2: 'RAIL3.SA',
 3: 'ALPA3.SA',
 4: 'ALPA4.SA',
 5: 'ALSO3.SA',
 6: 'AMAR3.SA',
 7: 'ABEV3.SA',
 8: 'ADHM3.SA',
 9: 'ARZZ3.SA',
 10: 'BBAS3.SA',
 11: 'BBDC3.SA',
 12: 'BBDC4.SA',
 13: 'BBRK3.SA',
 14: 'BEEF3.SA',
 15: 'BPHA3.SA',
 16: 'BPAN4.SA',
 17: 'BRAP3.SA',
 18: 'BRAP4.SA',
 19: 'BRFS3.SA'}

# 3. Calculando os descontos

O objetivo é encontrar os descontos dos ativos (diferença do preço de abertura para o preço atual). Dessa forma, podemos visualizar quais ativos possuem os menores riscos e os maiores descontos, assim o analista consegue decidir quais desses ativos ele irá realizar uma análise mais aprofunda.

**Função calculo do desconto**

In [11]:
def func_descont(name):
    """
    Essa função recebe o código da ação e retorna o desconto, risco e o último preço.
    """
    stock = stockDict[name]
    quantile75 = np.quantile(stock['Close'], .75)
    last_price = stock['Close'][-1]
    descont = (last_price - quantile75) / quantile75
    risk = np.std(stock['Close'])
    return descont, risk, last_price

**Coletando as informações sobre os ativos**

In [12]:
# Criando um data frame
df = pd.DataFrame(columns = ['Descont', 'Risk', 'LastPrice'])

# Looping para coletar o desconto, risco e o último preço
for ticker in stockDict.keys():
    if len(stockDict[ticker]) != 0:
        descont, risk, last_price = func_descont(ticker)
        df.loc[ticker] = [descont, risk, last_price]
    
df.head()

,Descont,Risk,LastPrice
ABCB4.SA,0.007724,1.901196,17.940001
AGRO3.SA,-0.102655,3.242200,27.709999
RAIL3.SA,-0.108328,1.706771,17.799999
ALPA3.SA,-0.687344,11.762743,11.240000
ALPA4.SA,-0.709452,13.849071,12.450000


In [13]:
# Ordenando o dataframe em ordem descendente
df = df.sort_values(by = 'Descont', ascending = False)
df

,Descont,Risk,LastPrice
BBAS3.SA,0.120732,3.713049,39.799999
BEEF3.SA,0.011867,2.049024,13.430000
ABCB4.SA,0.007724,1.901196,17.940001
ADHM3.SA,0.000000,0.000000,1.560000
BPHA3.SA,0.000000,0.000000,0.620000
ARZZ3.SA,-0.061042,8.930518,84.410004
AGRO3.SA,-0.102655,3.242200,27.709999
RAIL3.SA,-0.108328,1.706771,17.799999
ABEV3.SA,-0.180364,1.324774,13.190000
ALSO3.SA,-0.285259,4.115334,17.940001


**Visualizando a relação risco x descontos dos ativos**

In [17]:
fig = px.scatter(
    df, 
    x = 'Risk',
    y = 'Descont', 
    size = 'LastPrice',
    hover_name = df.index,
    color='Risk'
)
fig.show()

# 4. Encontrando a carteira mais eficiente

Digamos que vamos escolher dois desses ativos para compor nossa carteira, porém não sabemos quanto devemos alocar em cada um para maximizar nossos. Neste caso podemos utilizar a técnica de otimização de carteira de Markwitz.

Como exemplo, vamos considerar que iremos comprar banco PAN e Bradesco.

**Criando um data frame com os preços de fechamento ajustados**

In [18]:
adj_close = pd.DataFrame({
    'BPAN4.SA': stockDict['BPAN4.SA']['Adj Close'], 
    'BBDC4.SA':stockDict['BBDC4.SA']['Adj Close']
}, 
    index = stockDict['BBDC4.SA'].index
)

adj_close.head()

,BPAN4.SA,BBDC4.SA
Date,,
2021-02-09 00:00:00-03:00,11.793015,18.868242
2021-02-10 00:00:00-03:00,11.477936,18.660650
2021-02-11 00:00:00-03:00,11.522947,18.682894
2021-02-12 00:00:00-03:00,11.306892,18.682894
2021-02-17 00:00:00-03:00,11.441926,18.668066


**Calculando o retorno**

In [19]:
retorno = adj_close.pct_change().dropna()
retorno

,BPAN4.SA,BBDC4.SA
Date,,
2021-02-10 00:00:00-03:00,-0.026717,-0.011002
2021-02-11 00:00:00-03:00,0.003922,0.001192
2021-02-12 00:00:00-03:00,-0.018750,0.000000
2021-02-17 00:00:00-03:00,0.011943,-0.000794
2021-02-18 00:00:00-03:00,0.019670,-0.009865
...,...,...
2023-02-02 00:00:00-03:00,-0.001701,0.015219
2023-02-03 00:00:00-03:00,-0.051107,-0.022238
2023-02-06 00:00:00-03:00,-0.012567,-0.003668


**Função para calcular o resultado da carteira anual**

In [20]:
def calcCarteira(df, peso):
    """
    Essa função calcula o retorno e a volatilidade anual da carteira
    """
    pesos = [peso, (1-peso)]
    df2 = df.dot(pesos).copy()
    return df2.mean() * 252, df2.std() * np.sqrt(252)

**Calculando os resultados da 100 carteiras**

In [21]:
carteiras = pd.DataFrame()
for i in np.linspace(0, 1, 101):
    media, volatilidade = calcCarteira (retorno, i)
    carteiras.loc[i, 'Retorno'] = media
    carteiras.loc[i, 'Volatilidade'] = volatilidade

carteiras.head()

,Retorno,Volatilidade
0.00,-0.090899,0.325748
0.01,-0.092308,0.324668
0.02,-0.093718,0.323689
0.03,-0.095128,0.322812
0.04,-0.096538,0.322038


In [22]:
menor_risco = carteiras['Volatilidade'].idxmin()

**Visualizando a fronteira eficiente de Markwitz**

In [23]:
fig = px.scatter(carteiras, x = 'Volatilidade', y = 'Retorno')
fig.show()

print(f'''A melhor alocação de carteira foi {menor_risco * 100}% em banco PAN e 
{(1-menor_risco) * 100}% Bradesco''')

A melhor alocação de carteira foi 11.0% em banco PAN e 
89.0% Bradesco
